# π Estimation with Monte Carlo methods
We demonstrate how to run Monte Carlo simulations with lithops. This notebook contains an example of estimation the number π with Monte Carlo. The goal of this notebook is to demonstrate how FaaS can benefit Monte Carlo simulations and not how it can be done using lithops.<br>
A Monte Carlo algorithm would randomly place points in the square and use the percentage of randomized points inside of the circle to estimate the value of π
![pi](https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif)
Requirements to run this notebook:

* AWS Cloud or GCP account. 
* You will need to have at least one existing object storage bucket. 

In [ ]:
from time import time
from random import random
import logging
import sys

import lithops

In [ ]:
MAP_INSTANCES = 100


class EstimatePI:
    randomize_per_map = 10000000

    def __init__(self):
        self.total_randomize_points = MAP_INSTANCES * self.randomize_per_map

    def __str__(self):
        return "Total Randomize Points: {:,}".format(self.randomize_per_map * MAP_INSTANCES)

    @staticmethod
    def predicate():
        x = random()
        y = random()
        return (x ** 2) + (y ** 2) <= 1

    def randomize_points(self, data):
        in_circle = 0
        for _ in range(self.randomize_per_map):
            in_circle += self.predicate()
        return float(in_circle / self.randomize_per_map)

    def process_in_circle_points(self, results):
        in_circle_percent = 0
        for map_result in results:
            in_circle_percent += map_result
        estimate_PI = float(4 * (in_circle_percent / MAP_INSTANCES))
        return estimate_PI

# Step 4 - Execute simulation with Lithops

In [ ]:
iterdata = [0] * MAP_INSTANCES
est_pi = EstimatePI()

start_time = time()
print("Monte Carlo simulation for estimating PI spawing over {} Cloud Function invocations".format(MAP_INSTANCES))
# obtain lithops executor
pw = lithops.FunctionExecutor(runtime_memory=2048)
# execute the code
pw.map_reduce(est_pi.randomize_points, iterdata, est_pi.process_in_circle_points)
#get results
result = pw.get_result()
elapsed = time()
print(str(est_pi))
print("Estimation of Pi: ", result)
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")